In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

#### 데이터 명
- 지하철 데이터 : KR_TB_SBWY_STN_HR_CON_20190101_20190531_UTF8.csv
- 철도 데이터 : KR_TB_TR_STN_DAY_CON_20190101_20190531_UTF8.csv

In [6]:
train_df = pd.read_csv('KR_TB_TR_STN_DAY_CON_20200101_20200531_UTF8.csv')
train_df.head()

,RUN_DT,STN_CD,STN_CD_NM,ABRD_PRNB,GOFF_PRNB
0,20200101,924,송도교,2,-
1,20200101,3900023,서울,"38,306","50,440"
2,20200101,3900025,용산,"19,702","23,967"
3,20200101,3900030,영등포,"10,812","11,125"
4,20200101,3900039,안양,387,269


필요한 날짜 지역 하차 데이터 사용

In [7]:
df = train_df.loc[:, ['RUN_DT', 'STN_CD_NM', ' GOFF_PRNB ']]
df.head()

,RUN_DT,STN_CD_NM,GOFF_PRNB
0,20200101,송도교,-
1,20200101,서울,"50,440"
2,20200101,용산,"23,967"
3,20200101,영등포,"11,125"
4,20200101,안양,269


In [8]:
df.isnull().sum()

RUN_DT         0
STN_CD_NM      0
 GOFF_PRNB     0
dtype: int64

In [9]:
df['STN_CD_NM'].unique()

array(['송도교', '서울', '용산', '영등포', '안양', '수원', '오산', '서정리', '평택', '성환',
       '천안', '전의', '조치원', '부강', '신탄진', '대전', '옥천', '이원', '지탄', '심천', '영동',
       '황간', '추풍령', '김천', '구미', '사곡', '약목', '왜관', '신동', '대구', '동대구', '경산',
       '남성현', '청도', '상동', '밀양', '삼랑진', '원동', '물금', '화명', '구포', '사상', '부산',
       '덕소', '양평', '용문', '지평', '석불', '일신', '매곡', '양동', '삼산', '동화', '만종',
       '원주', '반곡', '신림', '제천', '단양', '희방사', '풍기', '영주', '안동', '의성', '탑리',
       '화본', '신녕', '북영천', '영천', '건천', '서경주', '서대전', '계룡', '연산', '논산',
       '강경', '함열', '익산', '김제', '신태인', '정읍', '백양사', '장성', '광주송정', '나주',
       '다시', '함평', '무안', '몽탄', '일로', '임성리', '목포', '삼례', '전주', '임실', '오수',
       '남원', '곡성', '구례구', '순천', '여천', '여수엑스포', '오송', '청주', '오근장', '청주공항',
       '증평', '음성', '주덕', '충주', '삼탄', '아산', '온양온천', '도고온천', '신례원', '예산',
       '삽교', '홍성', '광천', '청소', '대천', '웅천', '판교', '서천', '장항', '군산', '대야',
       '행신', '옥수', '왕십리', '청량리', '평내호평', '청평', '가평', '백양리', '강촌', '남춘천',
       '춘천', '옥산', '청리', '상주', '함창', '점촌', '용궁', '개

In [24]:

for i in range(0, 266):
    df[' GOFF_PRNB '] = df[' GOFF_PRNB '].str.replace(",","")
    df.loc[df[' GOFF_PRNB ']==' - ', 'GOFF_PRNB'] = 0
    df[' GOFF_PRNB '].astype('float')
    
df['GOFF_PRNB']

ValueError: could not convert string to float: ' - '

In [20]:
stations = ['송도교', '서울', '용산', '영등포', '안양', '수원', '오산', '서정리', '평택', '성환',
       '천안', '전의', '조치원', '부강', '신탄진', '대전', '옥천', '이원', '심천', '영동', '황간',
       '추풍령', '김천', '구미', '사곡', '약목', '왜관', '신동', '대구', '동대구', '경산',
       '남성현', '청도', '상동', '밀양', '삼랑진', '원동', '물금', '화명', '구포', '사상', '부산',
       '덕소', '양평', '용문', '지평', '석불', '일신', '매곡', '양동', '삼산', '동화', '만종',
       '원주', '반곡', '신림', '제천', '단양', '희방사', '풍기', '영주', '안동', '의성', '탑리',
       '화본', '신녕', '북영천', '영천', '건천', '서경주', '서대전', '계룡', '연산', '논산',
       '강경', '함열', '익산', '김제', '신태인', '정읍', '백양사', '장성', '광주송정', '나주',
       '다시', '함평', '무안', '몽탄', '일로', '임성리', '목포', '삼례', '전주', '임실', '오수',
       '남원', '곡성', '구례구', '순천', '여천', '여수엑스포', '오송', '청주', '오근장', '청주공항',
       '증평', '음성', '주덕', '충주', '삼탄', '아산', '온양온천', '신창', '도고온천', '신례원',
       '예산', '삽교', '홍성', '광천', '청소', '대천', '웅천', '판교', '서천', '장항', '군산',
       '대야', '행신', '옥수', '왕십리', '청량리', '동두천', '소요산', '전곡', '연천', '신망리',
       '대광리', '신탄리', '평내호평', '청평', '가평', '백양리', '강촌', '남춘천', '춘천', '옥산',
       '청리', '상주', '함창', '점촌', '용궁', '개포', '예천', '극락강', '광주', '봉화', '춘양',
       '현동', '분천', '양원', '승부', '석포', '철암', '동백산', '도계', '신기', '동해', '묵호',
       '망상해변', '정동진', '강릉', '쌍룡', '영월', '예미', '민둥산', '사북', '고한', '태백',
       '부전', '센텀', '신해운대', '기장', '좌천', '월내', '남창', '덕하', '태화강', '호계',
       '불국사', '경주', '안강', '하양', '한림정', '진영', '창원', '마산', '중리', '함안', '군북',
       '반성', '진주', '완사', '북천', '횡천', '하동', '진상', '광양', '벌교', '조성', '예당',
       '득량', '보성', '명봉', '이양', '능주', '화순', '효천', '서광주', '광명', '천안아산',
       '김천구미', '신경주', '울산', '상봉', '진례', '창원중앙', '인천공항T1', '백마고지', '공주',
       '포항', '횡성', '둔내', '평창', '진부', '인천공항T2', '월포', '장사', '강구', '영덕',
       '지탄', '각계', '문산', '임진강', '도라산', '초성리', '퇴계원', '사릉', '마석', '임기',
       '별어곡', '선평', '정선', '아우라지', '비동', '광운대', '의정부', '한탄강', '나전', '운천',
       '서빙고', '망우', '백운', '자미원', '의왕', '창동', '별내']

for i in range(0, 266):
    station = df[df['STN_CD_NM'] == stations[i]]
    station['GOFF_PRNB'] = station[' GOFF_PRNB '].str.replace(",","")
    station.loc[station['GOFF_PRNB']==' - ', 'GOFF_PRNB'] = 0
    station['GOFF_PRNB'].astype('float')
    
    print(stations[i], '\n', station['GOFF_PRNB'], '\n')
station['GOFF_PRNB'].info()

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


송도교 
 0          0
468        0
706       1 
1871      2 
2106       0
2342       0
2576       0
2812       0
3514      2 
3751      1 
3989      2 
4224      2 
4460      2 
4929      2 
5640       0
5873       0
6575      2 
6812       0
8684      1 
8916       0
9148      1 
9615      1 
10314     1 
11475     2 
12866     2 
27711      0
Name: GOFF_PRNB, dtype: object 

서울 
 1         50440 
233       49116 
469       58289 
707       51642 
939       51119 
          ...   
33894     29088 
34125     32026 
34354     39010 
34585     25796 
34813     29179 
Name: GOFF_PRNB, Length: 152, dtype: object 

용산 
 2         23967 
234       19958 
470       24878 
708       27122 
940       25421 
          ...   
33895     12808 
34126     13757 
34355     18114 
34586     15721 
34814     17829 
Name: GOFF_PRNB, Length: 152, dtype: object 

영등포 
 3         11125 
235       10732 
471       13173 
709       14459 
941       13150 
          ...   
33896      7616 
34127      8190 
34356

양평 
 44        1019 
277       1373 
513       1655 
749       1715 
982       1355 
          ...  
33937     1079 
34166     1141 
34397     1516 
34628     1110 
34855      764 
Name: GOFF_PRNB, Length: 152, dtype: object 

용문 
 45        248 
278       346 
514       401 
750       408 
983       364 
         ...  
33938     283 
34167     287 
34398     335 
34629     348 
34856     238 
Name: GOFF_PRNB, Length: 152, dtype: object 

지평 
 46        33 
279       49 
515       70 
751       30 
984       35 
         ... 
33939     45 
34168     44 
34399     53 
34630     27 
34857     11 
Name: GOFF_PRNB, Length: 152, dtype: object 

석불 
 47        1 
280        0
516       2 
752       1 
1217      3 
        ... 
33940     4 
34169     6 
34400     4 
34631     5 
34858     4 
Name: GOFF_PRNB, Length: 150, dtype: object 

일신 
 48        12 
281        9 
517        7 
753       13 
985        7 
         ... 
33941     17 
34170     19 
34401     20 
34632     30 
34859     17 

목포 
 91        2364 
324       2312 
560       2781 
796       2952 
1028      2520 
          ...  
33984     1632 
34213     1741 
34444     2313 
34675     2109 
34902     1560 
Name: GOFF_PRNB, Length: 152, dtype: object 

삼례 
 92        167 
325       142 
561       155 
797       129 
1029      167 
         ...  
33985      86 
34214      83 
34445     132 
34676     125 
34903     203 
Name: GOFF_PRNB, Length: 152, dtype: object 

전주 
 93        4374 
326       3732 
562       4670 
798       5725 
1030      4191 
          ...  
33986     2585 
34215     2722 
34446     4047 
34677     4419 
34904     2964 
Name: GOFF_PRNB, Length: 152, dtype: object 

임실 
 94         89 
327        99 
563       121 
799        99 
1031      122 
         ...  
33987      64 
34216      98 
34447      88 
34678      98 
34905      92 
Name: GOFF_PRNB, Length: 152, dtype: object 

오수 
 95         89 
328        84 
564       104 
800       114 
1032      107 
         ...  
33988      82 
3421

강촌 
 135       133 
371        29 
607        33 
841       159 
1072       74 
         ...  
34030      10 
34259      13 
34490      12 
34719     178 
34946     156 
Name: GOFF_PRNB, Length: 152, dtype: object 

남춘천 
 136       2243 
372       2055 
608       2536 
842       3517 
1073      2584 
          ...  
34031     1624 
34260     1573 
34491     2348 
34720     2493 
34947     1689 
Name: GOFF_PRNB, Length: 152, dtype: object 

춘천 
 137       1964 
373       1732 
609       1961 
843       2922 
1074      2176 
          ...  
34032     1190 
34261     1255 
34492     1680 
34721     1997 
34948     1358 
Name: GOFF_PRNB, Length: 152, dtype: object 

옥산 
 138       21 
374       31 
610       34 
844       23 
1075      26 
         ... 
34033     22 
34262     13 
34493     26 
34722     26 
34949     20 
Name: GOFF_PRNB, Length: 152, dtype: object 

청리 
 139        4 
375        7 
611       11 
845        9 
1076      12 
         ... 
34034      6 
34263      5 
34494  

경주 
 184       1050 
419       1359 
657       1396 
890       1409 
1121      1267 
          ...  
34076      833 
34305      795 
34536      979 
34764     1232 
34991      876 
Name: GOFF_PRNB, Length: 152, dtype: object 

안강 
 185       37 
420       24 
658       22 
891       45 
1122      31 
         ... 
34077     11 
34306     13 
34537     35 
34765     29 
34992     23 
Name: GOFF_PRNB, Length: 152, dtype: object 

하양 
 186       856 
421       784 
659       783 
892       755 
1123      929 
         ...  
34078     442 
34307     410 
34538     556 
34766     533 
34993     699 
Name: GOFF_PRNB, Length: 152, dtype: object 

한림정 
 187       28 
422       22 
660       20 
893       26 
1124       7 
         ... 
34079     16 
34308     19 
34539     16 
34767     35 
34994     30 
Name: GOFF_PRNB, Length: 152, dtype: object 

진영 
 188       702 
423       509 
661       709 
894       718 
1125      844 
         ...  
34080     297 
34309     374 
34540     526 
34768 

강구 
 231       43 
466       29 
704       30 
937       41 
1168      26 
         ... 
34123     15 
34352     24 
34583     24 
34811     23 
35038      9 
Name: GOFF_PRNB, Length: 152, dtype: object 

영덕 
 232       157 
467       176 
705       189 
938       184 
1169      153 
         ...  
34124     105 
34353     100 
34584     126 
34812     125 
35039     119 
Name: GOFF_PRNB, Length: 152, dtype: object 

지탄 
 18        1 
250       2 
486       3 
956        0
1187      2 
        ... 
33682      0
33911      0
34371     2 
34602     2 
34830      0
Name: GOFF_PRNB, Length: 125, dtype: object 

각계 
 252      0
488      0
1189     0
1891     0
3065     0
        ..
32538    0
33453    0
33913    0
34373    0
34604    0
Name: GOFF_PRNB, Length: 66, dtype: object 

문산 
 Series([], Name: GOFF_PRNB, dtype: object) 

임진강 
 Series([], Name: GOFF_PRNB, dtype: object) 

도라산 
 Series([], Name: GOFF_PRNB, dtype: object) 

초성리 
 Series([], Name: GOFF_PRNB, dtype: object) 

퇴계원 
 364  

AttributeError: 'Series' object has no attribute 'info'

In [18]:
station.head()

,RUN_DT,STN_CD_NM,GOFF_PRNB,GOFF_PRNB


In [ ]:
print(station.dtypes)